In [ ]:
!pip install rouge-score sentence-transformers


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3833f713d65781d10dde4a5fd647a1afe59e99786f9c91bc382d563085bc8265
  

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt

# Load models
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Small dataset
documents = [
    [
        "The stock market crashed on Monday.",
        "Investors were worried about inflation.",
        "Tech stocks led the losses.",
        "Analysts predict a slow recovery.",
        "Some investors see buying opportunities."
    ],
    [
        "The hurricane caused severe flooding.",
        "Thousands were evacuated from coastal areas.",
        "Emergency crews rescued stranded residents.",
        "Roads and bridges were damaged.",
        "Authorities declared a state of emergency."
    ]
]

# Generate realistic summaries
generated_summaries = []
for doc in documents:
    input_text = " ".join(doc)
    inputs = tokenizer([input_text], max_length=1024, return_tensors="pt", truncation=True)
    summary_ids = bart_model.generate(
        inputs["input_ids"],
        num_beams=4,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        max_length=60,
        min_length=20,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

# Print generated summaries
for i, summary in enumerate(generated_summaries):
    print(f"\nGenerated Summary {i+1}:\n{summary}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Generated Summary 1:
The stock market crashed on Monday. Tech stocks led the losses. Analysts predict a slow recovery. Some investors see buying opportunities.

Generated Summary 2:
The hurricane caused severe flooding. Thousands were evacuated from coastal areas. Roads and bridges were damaged.


In [ ]:
# Compute ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
for i, summary in enumerate(generated_summaries):
    reference = " ".join(documents[i])
    score = scorer.score(reference, summary)
    print(f"\nDocument {i+1} ROUGE Scores:")
    for metric in score:
        print(f"{metric.upper()} - Precision: {score[metric].precision:.3f}, "
              f"Recall: {score[metric].recall:.3f}, F1: {score[metric].fmeasure:.3f}")

# Cosine similarity between centroid and summary
print("\nCentroid vs Summary Similarity (cosine):")
for i, doc in enumerate(documents):
    doc_embed = embedder.encode(doc)
    centroid = np.mean(doc_embed, axis=0)
    summary_embed = embedder.encode([generated_summaries[i]])[0]
    sim = cosine_similarity([centroid], [summary_embed])[0][0]
    print(f"Document {i+1}: {sim:.4f}")



Document 1 ROUGE Scores:
ROUGE1 - Precision: 1.000, Recall: 0.808, F1: 0.894
ROUGE2 - Precision: 0.950, Recall: 0.760, F1: 0.844
ROUGEL - Precision: 1.000, Recall: 0.808, F1: 0.894

Document 2 ROUGE Scores:
ROUGE1 - Precision: 1.000, Recall: 0.593, F1: 0.744
ROUGE2 - Precision: 0.933, Recall: 0.538, F1: 0.683
ROUGEL - Precision: 1.000, Recall: 0.593, F1: 0.744

Centroid vs Summary Similarity (cosine):
Document 1: 0.8201
Document 2: 0.8352
